<a href="https://colab.research.google.com/github/Moksh45/Host-your-AI-App-on-Google-Colab/blob/main/How_to_Host_your_AI_App_on_Google_Colab_for_Free.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pciutils and Ollama
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (327 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: un

In [2]:
# Install pinggy
!pip install pinggy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 52.4 MB/s eta 0:00:00


In [3]:
import subprocess
import os

def start_ollama_server():
    # Use nohup to run ollama serve in the background and redirect output to a file
    subprocess.Popen(['nohup', 'ollama', 'serve'],
                     stdout=open('ollama.log', 'w'),
                     stderr=open('ollama_error.log', 'w'),
                     preexec_fn=os.setsid)
    print("🚀 Ollama server launched successfully in the background!")

# Start the Ollama server in the background
start_ollama_server()

🚀 Ollama server launched successfully in the background!


In [4]:
# Check if Ollama is listening on port 11434
def check_ollama_port(port='11434'):
    try:
        output = subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'],
                              capture_output=True, text=True).stdout
        if f":{port} (LISTEN)" in output:
            print(f"✅ Ollama is actively listening on port {port}")
        else:
            print(f"⚠️ Ollama not detected on port {port}")
    except Exception as e:
        print(f"❌ Error checking port: {e}")
check_ollama_port()

✅ Ollama is actively listening on port 11434


In [5]:
#install llama model
!ollama pull llama3.2:1b

In [6]:
import pinggy
tunnel1 = pinggy.start_tunnel(
    forwardto="localhost:8000",
    headermodification=["u:Host:localhost:5000"]
)


print(f"Tunnel1 started - URLs: {tunnel1.urls}")

Tunnel1 started - URLs: ['http://cusyh-104-198-104-111.a.free.pinggy.link', 'https://cusyh-104-198-104-111.a.free.pinggy.link']


In [8]:
from flask import Flask, request, render_template_string
import requests
import json

app = Flask(__name__)

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Blog Writer</title>
    <style>
        body {
            font-family: 'Inter', system-ui, sans-serif;
            background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%);
            margin: 0;
            padding: 0;
            color: #333;
            min-height: 100vh;
            display: flex;
            align-items: center;
            justify-content: center;
        }
        .container {
            background: #fff;
            padding: 2.5rem;
            border-radius: 20px;
            box-shadow: 0 15px 40px rgba(0,0,0,0.2);
            width: 90%;
            max-width: 800px;
            animation: fadeIn 0.6s ease;
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }
        h1 {
            text-align: center;
            color: #222;
            margin-bottom: 1.5rem;
            font-size: 2.2rem;
        }
        form {
            display: flex;
            gap: 1rem;
        }
        input[type=text] {
            flex: 1;
            padding: 0.9rem;
            border-radius: 10px;
            border: 1px solid #ccc;
            font-size: 1rem;
        }
        button {
            padding: 0.9rem 1.5rem;
            background-color: #2575fc;
            border: none;
            border-radius: 10px;
            color: white;
            font-weight: bold;
            font-size: 1rem;
            cursor: pointer;
            transition: 0.3s;
        }
        button:hover {
            background-color: #1b5edc;
        }
        .loading {
            text-align: center;
            margin-top: 2rem;
            font-style: italic;
            color: #666;
        }
        .blog {
            margin-top: 2.5rem;
            background: #fafafa;
            padding: 1.5rem;
            border-radius: 12px;
            line-height: 1.7;
            font-size: 1.05rem;
            box-shadow: inset 0 0 5px rgba(0,0,0,0.05);
        }
        .blog h2 {
            color: #2575fc;
            margin-bottom: 1rem;
        }
        .footer {
            text-align: center;
            margin-top: 2rem;
            font-size: 0.9rem;
            color: #777;
        }
    </style>
    <script>
        function showLoader() {
            document.getElementById('loading').style.display = 'block';
        }
    </script>
</head>
<body>
    <div class="container">
        <h1>📝 AI Blog Writer</h1>
        <form method="post" onsubmit="showLoader()">
            <input type="text" name="title" placeholder="Enter your blog topic..." required value="{{ title or '' }}">
            <button type="submit">Generate</button>
        </form>

        <div id="loading" class="loading" style="display:none;">
            🚀 Generating your blog... please wait.
        </div>

        {% if blog %}
            <div class="blog">
                <h2>{{ title }}</h2>
                <p>{{ blog }}</p>
            </div>
        {% endif %}

        <div class="footer">
            Powered by <strong>Ollama</strong> • Designed by <strong>Pinggy Labs</strong>
        </div>
    </div>
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    blog = None
    title = None

    if request.method == "POST":
        title = request.form.get("title")
        prompt = f"Write a detailed and engaging blog post about: '{title}'."

        try:
            response = requests.post(
                "http://localhost:11434/api/generate",
                json={"model": "llama3.2:1b", "prompt": prompt},
                stream=True,
                timeout=120
            )

            blog_parts = []
            for line in response.iter_lines():
                if line:
                    try:
                        data = json.loads(line.decode("utf-8"))
                        if "response" in data:
                            blog_parts.append(data["response"])
                    except json.JSONDecodeError:
                        continue

            blog = "".join(blog_parts).strip()

        except Exception as e:
            blog = f"⚠️ Error: Could not reach Ollama API ({e})"

    return render_template_string(HTML_TEMPLATE, blog=blog, title=title)


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Oct/2025 04:12:47] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2025 04:13:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2025 04:13:22] "GET / HTTP/1.1" 200 -
